<a href="https://colab.research.google.com/github/WaliMuhammadAhmad/CODEX-Dataset/blob/main/Dataset_Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import os
import shutil
import tarfile

from datasets import Dataset, load_dataset
from huggingface_hub import login, Repository
from google.colab import userdata

In [4]:
login(userdata.get('HF_TOKEN'),add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
dir = "raw_dataset"

repo_url="CodexAI/raw_dataset"

repo_type="dataset"

# Dataset Compression
In this section:

1. Clone the already existing `dataset` repository from hf.
2. Check if the cloned repository consists of folders or not.
3. If folders then compress these folders to `tar.bz2` format.
4. Commit the changes onto hf repository.

**Note**: These steps uses the `huggingface_hub.Repository` method which, according to huggingface soon will be deprecated, for the time begin, It's working and i am using it.
*Learn more here* : https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http


In [15]:
repo = Repository(local_dir=dir,clone_from=repo_url,repo_type=repo_type)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/datasets/CodexAI/raw_dataset into local empty directory.


In [16]:
repo.git_pull()

In [17]:
def archive_folders(root_dir):

    for folder in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, folder)
        print(f"Processing {folder_path}...")

        if os.path.isdir(folder_path):

            tar_name = f"{folder}.tar.bz2"
            tar_path = os.path.join(root_dir, tar_name)


            with tarfile.open(tar_path, "w:bz2") as tar:

                tar.add(folder_path, arcname=os.path.basename(folder_path))

            print(f"Finished: {folder_path} -> {tar_path}")

            # shutil.rmtree(folder_path)
            # print(f"Deleted: {folder_path}")

In [18]:
archive_folders(dir)

Processing raw_dataset/train...
Finished: raw_dataset/train -> raw_dataset/train.tar.bz2
Processing raw_dataset/.gitattributes...
Processing raw_dataset/.git...
Finished: raw_dataset/.git -> raw_dataset/.git.tar.bz2
Processing raw_dataset/test...
Finished: raw_dataset/test -> raw_dataset/test.tar.bz2
Processing raw_dataset/eval...
Finished: raw_dataset/eval -> raw_dataset/eval.tar.bz2


In [22]:
# shutil.rmtree('/content/raw_dataset/train')

In [23]:
!rm raw_dataset/.git.tar.bz2

In [24]:
os.listdir(dir)

['eval.tar.bz2', '.gitattributes', 'train.tar.bz2', '.git', 'test.tar.bz2']

In [25]:
repo.push_to_hub(commit_message=f"Compressed dataset uploaded")

Upload file train.tar.bz2:   0%|          | 1.00/35.3M [00:00<?, ?B/s]

Upload file test.tar.bz2:   0%|          | 1.00/15.7M [00:00<?, ?B/s]

Upload file eval.tar.bz2:   0%|          | 1.00/15.7M [00:00<?, ?B/s]

To https://huggingface.co/datasets/CodexAI/raw_dataset
   78c853f3b..87d1b33a4  main -> main

   78c853f3b..87d1b33a4  main -> main



'https://huggingface.co/datasets/CodexAI/raw_dataset/commit/87d1b33a42b06d3f7a3fb363f72524d1c8c7e878'

# Load Compressed Dataset
In this section:

1. Load the already existing `dataset` from hf which was compressed into `tar.bz2`.
2. Play with dataset.

In [26]:
dataset = load_dataset(repo_url)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [27]:
dataset

DatasetDict({
    train: Dataset({
        features: ['__key__', '__url__', 'json'],
        num_rows: 62813
    })
    test: Dataset({
        features: ['__key__', '__url__', 'json'],
        num_rows: 41874
    })
})

In [28]:
dataset['train']

Dataset({
    features: ['__key__', '__url__', 'json'],
    num_rows: 62813
})

In [35]:
dataset['test']

Dataset({
    features: ['__key__', '__url__', 'json'],
    num_rows: 41874
})

In [36]:
dataset['test'].features

{'__key__': Value(dtype='string', id=None),
 '__url__': Value(dtype='string', id=None),
 'json': {'focal_class': {'fields': [{'declarator': Value(dtype='string', id=None),
     'modifier': Value(dtype='string', id=None),
     'original_string': Value(dtype='string', id=None),
     'type': Value(dtype='string', id=None),
     'var_name': Value(dtype='string', id=None)}],
   'file': Value(dtype='string', id=None),
   'identifier': Value(dtype='string', id=None),
   'interfaces': Value(dtype='string', id=None),
   'methods': [{'class_method_signature': Value(dtype='string', id=None),
     'constructor': Value(dtype='bool', id=None),
     'full_signature': Value(dtype='string', id=None),
     'identifier': Value(dtype='string', id=None),
     'modifiers': Value(dtype='string', id=None),
     'parameters': Value(dtype='string', id=None),
     'return': Value(dtype='string', id=None),
     'signature': Value(dtype='string', id=None),
     'testcase': Value(dtype='bool', id=None)}],
   'super

In [37]:
dataset['test']['__key__']

['test/1/100402355_1',
 'test/1/100402355_101',
 'test/1/100402355_110',
 'test/1/100402355_121',
 'test/1/100402355_123',
 'test/1/100402355_132',
 'test/1/100402355_134',
 'test/1/100402355_135',
 'test/1/100402355_140',
 'test/1/100402355_147',
 'test/1/100402355_158',
 'test/1/100402355_22',
 'test/1/100402355_29',
 'test/1/100402355_3',
 'test/1/100402355_32',
 'test/1/100402355_33',
 'test/1/100402355_4',
 'test/1/100402355_40',
 'test/1/100402355_42',
 'test/1/100402355_46',
 'test/1/100402355_47',
 'test/1/100402355_49',
 'test/1/100402355_51',
 'test/1/100402355_59',
 'test/1/100402355_61',
 'test/1/100402355_72',
 'test/1/100402355_75',
 'test/1/100402355_76',
 'test/1/100402355_82',
 'test/1/100402355_84',
 'test/1/100402355_85',
 'test/1/100402355_88',
 'test/1/100402355_9',
 'test/1/100402355_91',
 'test/1/1011539_12',
 'test/1/1011539_15',
 'test/1/1011539_28',
 'test/1/1011539_30',
 'test/1/1011539_31',
 'test/1/1011539_37',
 'test/1/1011539_7',
 'test/1/103560333_1',
 '

In [38]:
dataset['test'][0]

{'__key__': 'test/1/100402355_1',
 '__url__': '/root/.cache/huggingface/datasets/downloads/e09c60e95ebc4ea975934aa879d5f4813f719962953dc05abe4f7ab90f2bed68',
 'json': {'focal_class': {'fields': [{'declarator': 'LOG = LoggerFactory.getLogger(GoalOptimizer.class)',
     'modifier': 'private static final',
     'original_string': 'private static final Logger LOG = LoggerFactory.getLogger(GoalOptimizer.class);',
     'type': 'Logger',
     'var_name': 'LOG'},
    {'declarator': 'HALF_MINUTE_IN_MS = TimeUnit.SECONDS.toMillis(30)',
     'modifier': 'private static final',
     'original_string': 'private static final long HALF_MINUTE_IN_MS = TimeUnit.SECONDS.toMillis(30);',
     'type': 'long',
     'var_name': 'HALF_MINUTE_IN_MS'},
    {'declarator': '_goalsByPriority',
     'modifier': 'private final',
     'original_string': 'private final List<Goal> _goalsByPriority;',
     'type': 'List<Goal>',
     'var_name': '_goalsByPriority'},
    {'declarator': '_balancingConstraint',
     'modifi

In [39]:
shutil.rmtree(dir)